In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
import optuna

# Load the dataset
breast = datasets.load_breast_cancer()
X = breast.data
y = breast.target

# Standardize the features for better performance
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE for oversampling the minority class
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Define the objective function for Optuna optimization
def objective(trial):
    classifier_name = trial.suggest_categorical('classifier', ['SVM', 'Random Forest', 'KNN'])

    if classifier_name == 'SVM':
        svc_c = trial.suggest_loguniform('svc_c', 0.1, 10)
        classifier_obj = SVC(C=svc_c)
    elif classifier_name == 'Random Forest':
        rf_n_estimators = trial.suggest_int('rf_n_estimators', 100, 500, step=100)
        classifier_obj = RandomForestClassifier(n_estimators=rf_n_estimators)
    else:
        knn_n_neighbors = trial.suggest_int('knn_n_neighbors', 3, 10)
        classifier_obj = KNeighborsClassifier(n_neighbors=knn_n_neighbors)

    pipeline = Pipeline([
        ('model', classifier_obj)
    ])

    pipeline.fit(X_train_smote, y_train_smote)

    return pipeline.score(X_test, y_test)

# Run Optuna optimization
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=100)

# Get the best model and its parameters
best_model_name = study.best_trial.user_attrs['classifier_name']
best_model_params = study.best_params

# Train the best model on the entire training set
best_model = None

if best_model_name == 'SVM':
    best_model = SVC(C=best_model_params['svc_c'])
elif best_model_name == 'Random Forest':
    best_model = RandomForestClassifier(n_estimators=best_model_params['rf_n_estimators'])
else:
    best_model = KNeighborsClassifier(n_neighbors=best_model_params['knn_n_neighbors'])

best_model.fit(X_train_smote, y_train_smote)

# Evaluate the best model on the test set
y_pred = best_model.predict(X_test)

# Print the classification report
print(f'Best Model: {best_model_name}')
print('Classification Report:')
print(classification_report(y_test, y_pred))
